In [18]:
import os
import sys
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import pprint

In [19]:
SPOTIPY_CLIENT_ID = 'd7e917f50adf456d9ac5c0342ca84fa0'
SPOTIPY_CLIENT_SECRET = '6b8d07dde47641adb46585a16d4fd024'
SPOTIPY_REDIRECT_URI = 'http://localhost:8888/callback'
scope = "user-library-read"

In [20]:
#Authentication - without user
client_credentials_manager = SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID,
                                                      client_secret=SPOTIPY_CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [21]:
name = 'Ayra Starr'
result = sp.search(name) #search query
result['tracks']['items'][0]['artists']

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/3ZpEKRjHaHANcpk10u6Ntq'},
  'href': 'https://api.spotify.com/v1/artists/3ZpEKRjHaHANcpk10u6Ntq',
  'id': '3ZpEKRjHaHANcpk10u6Ntq',
  'name': 'Ayra Starr',
  'type': 'artist',
  'uri': 'spotify:artist:3ZpEKRjHaHANcpk10u6Ntq'}]

In [22]:
# Extract Artist uri
artist_uri =  result['tracks']['items'][0]['artists'][0]['uri']
artist_uri

'spotify:artist:3ZpEKRjHaHANcpk10u6Ntq'

In [23]:
sp_albums = sp.artist_albums(artist_uri, album_type='album')
    # Storing album names and URI's
album_names = []
album_uris = []
for i in range(len(sp_albums['items'])):
    album_names.append(sp_albums['items'][i]['name'])
    album_uris.append(sp_albums['items'][i]['uri'])
    album_uris.append(sp_albums['items'][i]['uri'])

album_names, album_uris    

(['19 & Dangerous (Deluxe)', '19 & Dangerous'],
 ['spotify:album:16ppCNm1KGCgUS0g3iKqh8',
  'spotify:album:16ppCNm1KGCgUS0g3iKqh8',
  'spotify:album:0AjdvP8p42lwSzmN0PpwJv',
  'spotify:album:0AjdvP8p42lwSzmN0PpwJv'])

In [24]:
album_uris,album_names

(['spotify:album:16ppCNm1KGCgUS0g3iKqh8',
  'spotify:album:16ppCNm1KGCgUS0g3iKqh8',
  'spotify:album:0AjdvP8p42lwSzmN0PpwJv',
  'spotify:album:0AjdvP8p42lwSzmN0PpwJv'],
 ['19 & Dangerous (Deluxe)', '19 & Dangerous'])

In [25]:
def albumSongs(uri):
    album = uri #assign album uri to a_name
    spotify_albums[album] = {} #Creates dictionary for that specific album
    #Create keys-values of empty lists inside nested dictionary for album
    spotify_albums[album]['album'] = [] #create empty list
    spotify_albums[album]['track_number'] = []
    spotify_albums[album]['id'] = []
    spotify_albums[album]['name'] = []
    spotify_albums[album]['uri'] = []
    
    tracks = sp.album_tracks(album) #pull data on album tracks
    for n in range(len(tracks['items'])): #for each song track
        spotify_albums[album]['album'].append(album_names[album_count]) #append album name tracked via album_count
        spotify_albums[album]['track_number'].append(tracks['items'][n]['track_number'])
        spotify_albums[album]['id'].append(tracks['items'][n]['id'])
        spotify_albums[album]['name'].append(tracks['items'][n]['name'])
        spotify_albums[album]['uri'].append(tracks['items'][n]['uri'])

In [26]:
spotify_albums = {}
album_count = 0
for i in album_uris: #each album
    albumSongs(i)
    print("Album " + str(album_names[album_count]) + " songs has been added to spotify_albums dictionary")
   #album_count+=1 #Updates album count once all tracks have been added

Album 19 & Dangerous (Deluxe) songs has been added to spotify_albums dictionary
Album 19 & Dangerous (Deluxe) songs has been added to spotify_albums dictionary
Album 19 & Dangerous (Deluxe) songs has been added to spotify_albums dictionary
Album 19 & Dangerous (Deluxe) songs has been added to spotify_albums dictionary


In [27]:
def audio_features(album):
    #Add new key-values to store audio features
    spotify_albums[album]['acousticness'] = []
    spotify_albums[album]['danceability'] = []
    spotify_albums[album]['energy'] = []
    spotify_albums[album]['instrumentalness'] = []
    spotify_albums[album]['liveness'] = []
    spotify_albums[album]['loudness'] = []
    spotify_albums[album]['speechiness'] = []
    spotify_albums[album]['tempo'] = []
    spotify_albums[album]['valence'] = []
    spotify_albums[album]['popularity'] = []
    #create a track counter
    track_count = 0
    for track in spotify_albums[album]['uri']:
        #pull audio features per track
        features = sp.audio_features(track)
        
        #Append to relevant key-value
        spotify_albums[album]['acousticness'].append(features[0]['acousticness'])
        spotify_albums[album]['danceability'].append(features[0]['danceability'])
        spotify_albums[album]['energy'].append(features[0]['energy'])
        spotify_albums[album]['instrumentalness'].append(features[0]['instrumentalness'])
        spotify_albums[album]['liveness'].append(features[0]['liveness'])
        spotify_albums[album]['loudness'].append(features[0]['loudness'])
        spotify_albums[album]['speechiness'].append(features[0]['speechiness'])
        spotify_albums[album]['tempo'].append(features[0]['tempo'])
        spotify_albums[album]['valence'].append(features[0]['valence'])
        #popularity is stored elsewhere
        pop = sp.track(track)
        spotify_albums[album]['popularity'].append(pop['popularity'])
        track_count+=1

In [28]:
import time
import numpy as np
sleep_min = 2
sleep_max = 5
start_time = time.time()
request_count = 0
for i in spotify_albums:
    audio_features(i)
    request_count+=1
    if request_count % 5 == 0:
        print(str(request_count) + " playlists completed")
        time.sleep(np.random.uniform(sleep_min, sleep_max))
        print('Loop #: {}'.format(request_count))
        print('Elapsed Time: {} seconds'.format(time.time() - start_time))


In [29]:
dic_df = {}
dic_df['album'] = []
dic_df['track_number'] = []
dic_df['id'] = []
dic_df['name'] = []
dic_df['uri'] = []
dic_df['acousticness'] = []
dic_df['danceability'] = []
dic_df['energy'] = []
dic_df['instrumentalness'] = []
dic_df['liveness'] = []
dic_df['loudness'] = []
dic_df['speechiness'] = []
dic_df['tempo'] = []
dic_df['valence'] = []
dic_df['popularity'] = []
for album in spotify_albums: 
    for feature in spotify_albums[album]:
        dic_df[feature].extend(spotify_albums[album][feature])
        
len(dic_df['album'])
dic_df

{'album': ['19 & Dangerous (Deluxe)',
  '19 & Dangerous (Deluxe)',
  '19 & Dangerous (Deluxe)',
  '19 & Dangerous (Deluxe)',
  '19 & Dangerous (Deluxe)',
  '19 & Dangerous (Deluxe)',
  '19 & Dangerous (Deluxe)',
  '19 & Dangerous (Deluxe)',
  '19 & Dangerous (Deluxe)',
  '19 & Dangerous (Deluxe)',
  '19 & Dangerous (Deluxe)',
  '19 & Dangerous (Deluxe)',
  '19 & Dangerous (Deluxe)',
  '19 & Dangerous (Deluxe)',
  '19 & Dangerous (Deluxe)',
  '19 & Dangerous (Deluxe)',
  '19 & Dangerous (Deluxe)',
  '19 & Dangerous (Deluxe)',
  '19 & Dangerous (Deluxe)',
  '19 & Dangerous (Deluxe)',
  '19 & Dangerous (Deluxe)',
  '19 & Dangerous (Deluxe)',
  '19 & Dangerous (Deluxe)',
  '19 & Dangerous (Deluxe)',
  '19 & Dangerous (Deluxe)',
  '19 & Dangerous (Deluxe)',
  '19 & Dangerous (Deluxe)'],
 'track_number': [1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11],
 'id': ['36pyJOteVm6340Ie7WtlZu',
  '6jc3jO9Jq

In [30]:
df = pd.DataFrame.from_dict(dic_df)

In [31]:
final_df = df.drop_duplicates('name').sort_index()
final_df

,album,track_number,id,name,uri,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity
0,19 & Dangerous (Deluxe),1,36pyJOteVm6340Ie7WtlZu,Cast (Gen Z Anthem),spotify:track:36pyJOteVm6340Ie7WtlZu,0.3450,0.512,0.522,0.000006,0.1160,-8.374,0.3550,169.689,0.533,40
1,19 & Dangerous (Deluxe),2,6jc3jO9JqrbVphg8L03mNX,Fashion Killer,spotify:track:6jc3jO9JqrbVphg8L03mNX,0.2360,0.777,0.674,0.000390,0.1380,-8.998,0.1470,99.966,0.336,38
2,19 & Dangerous (Deluxe),3,2bowCgJxTfB2siFTKU1sRS,Ase,spotify:track:2bowCgJxTfB2siFTKU1sRS,0.3160,0.807,0.636,0.002960,0.1010,-7.844,0.0386,105.001,0.801,40
3,19 & Dangerous (Deluxe),4,6pz1iqWlBwYXiAzJB7OAZY,Bloody Samaritan (with Kelly Rowland) - Remix,spotify:track:6pz1iqWlBwYXiAzJB7OAZY,0.1250,0.806,0.738,0.000000,0.0805,-7.443,0.1490,105.008,0.385,44
4,19 & Dangerous (Deluxe),5,50QI8j3eYzwMEOUMLUw2jI,Lonely Refix,spotify:track:50QI8j3eYzwMEOUMLUw2jI,0.0666,0.752,0.627,0.003760,0.2420,-9.550,0.0406,114.995,0.465,46
5,19 & Dangerous (Deluxe),6,1tqoHiOZ6jKzO7cRALrjK7,Snitch,spotify:track:1tqoHiOZ6jKzO7cRALrjK7,0.1940,0.711,0.564,0.000000,0.1270,-8.143,0.0668,120.064,0.418,37
6,19 & Dangerous (Deluxe),7,3k0fZo6WeZcTdCAIXZPui6,Toxic,spotify:track:3k0fZo6WeZcTdCAIXZPui6,0.2250,0.711,0.576,0.000000,0.2040,-6.066,0.0983,139.947,0.292,38
7,19 & Dangerous (Deluxe),8,0B5FcqQEVmkqJ9OSdAgEVd,In Between,spotify:track:0B5FcqQEVmkqJ9OSdAgEVd,0.7540,0.636,0.444,0.000012,0.1090,-7.753,0.0299,104.973,0.154,36
8,19 & Dangerous (Deluxe),9,1tKsfYB65Kz74yk0HMCdcH,Running,spotify:track:1tKsfYB65Kz74yk0HMCdcH,0.5210,0.557,0.539,0.000000,0.1010,-9.587,0.0877,102.877,0.562,61
9,19 & Dangerous (Deluxe),10,5NVb7Jane3k2s5aw8UnA1A,Beggie Beggie,spotify:track:5NVb7Jane3k2s5aw8UnA1A,0.5720,0.723,0.430,0.000000,0.1010,-6.761,0.0429,99.995,0.324,38


In [32]:
result = sp.search("Tems")
track = result['tracks']['items'][0]

artist = sp.artist(track["artists"][0]["external_urls"]["spotify"])
print("artist genres:", artist["genres"])

album = sp.album(track["album"]["external_urls"]["spotify"])
print("album genres:", album["genres"])
print("album release-date:", album["release_date"])

artist genres: ['atl hip hop', 'hip hop', 'rap', 'southern hip hop', 'trap']
album genres: []
album release-date: 2022-04-29
